In [1]:
from google.colab import files
files.upload()  # Upload kaggle.json here

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ananyarachar","key":"ed9a2181fb870f1624c45696e52965ac"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the PlantVillage dataset
!kaggle datasets download -d mohitsingh1804/plantvillage
!unzip -q plantvillage.zip -d plantvillage


Dataset URL: https://www.kaggle.com/datasets/mohitsingh1804/plantvillage
License(s): GPL-2.0


In [3]:
!ls plantvillage/PlantVillage/train

 Apple___Apple_scab
 Apple___Black_rot
 Apple___Cedar_apple_rust
 Apple___healthy
 Blueberry___healthy
'Cherry_(including_sour)___healthy'
'Cherry_(including_sour)___Powdery_mildew'
'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot'
'Corn_(maize)___Common_rust_'
'Corn_(maize)___healthy'
'Corn_(maize)___Northern_Leaf_Blight'
 Grape___Black_rot
'Grape___Esca_(Black_Measles)'
 Grape___healthy
'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)'
'Orange___Haunglongbing_(Citrus_greening)'
 Peach___Bacterial_spot
 Peach___healthy
 Pepper,_bell___Bacterial_spot
 Pepper,_bell___healthy
 Potato___Early_blight
 Potato___healthy
 Potato___Late_blight
 Raspberry___healthy
 Soybean___healthy
 Squash___Powdery_mildew
 Strawberry___healthy
 Strawberry___Leaf_scorch
 Tomato___Bacterial_spot
 Tomato___Early_blight
 Tomato___healthy
 Tomato___Late_blight
 Tomato___Leaf_Mold
 Tomato___Septoria_leaf_spot
'Tomato___Spider_mites Two-spotted_spider_mite'
 Tomato___Target_Spot
 Tomato___Tomato_mosaic_virus
 Tomato_

In [4]:
!pip install qiskit==0.44.1 qiskit-aer==0.12.0 qiskit-machine-learning==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 10.4 MB/s eta 0:00:00
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp311-cp311-linux_x86_64.whl size=542086 sha256=be581bb755cd53a044437175572917fb8d0372d0f0b4b8f76c8ca2dbc0f189c6
  Stored in directory: /root/.cache/pip/wheels/5c/8a/f6/fd3df9a9714677410a5ccbf3c

In [5]:
!pip install opencv-python-headless scikit-learn tqdm qiskit qiskit-machine-learning

In [6]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Step 1: Load and preprocess the dataset
selected_classes = ['Tomato___Bacterial_spot', 'Tomato___healthy',
                    'Strawberry___Leaf_scorch', 'Corn_(maize)___Common_rust_',
                    'Peach___Bacterial_spot', 'Peach___healthy', 'Potato___healthy']
image_dir = 'plantvillage/PlantVillage/train'

X, y = [], []

for class_name in selected_classes:
    class_path = os.path.join(image_dir, class_name)
    for img_name in tqdm(os.listdir(class_path)[:100]):  # Limit to 100 images per class
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (64, 64))
            hist = cv2.calcHist([img], [0, 1, 2], None, [4, 4, 4],
                                [0, 256, 0, 256, 0, 256])
            X.append(hist.flatten())
            y.append(class_name)

X = np.array(X)
y = np.array(y)
print(f"Feature shape: {X.shape}, Labels: {np.unique(y)}")

# Step 2: Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Step 3: Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42)

# Step 4: Train SVC
model = SVC(kernel='linear')
model.fit(X_train, y_train)

# Step 5: Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy * 100:.2f}%")

print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Step 6: Save model and encoder in .pkl format
joblib.dump(model, 'qml_model.pkl')
joblib.dump(label_encoder, 'q_layer.pkl')

print("\nModel saved as qml_model.pkl")
print("Label encoder saved as q_layer.pkl")


100%|██████████| 100/100 [00:00<00:00, 1384.00it/s]


Feature shape: (700, 64), Labels: ['Corn_(maize)___Common_rust_' 'Peach___Bacterial_spot' 'Peach___healthy'
 'Potato___healthy' 'Strawberry___Leaf_scorch' 'Tomato___Bacterial_spot'
 'Tomato___healthy']

Accuracy: 91.43%

Classification Report:
                              precision    recall  f1-score   support

Corn_(maize)___Common_rust_       0.95      1.00      0.98        20
     Peach___Bacterial_spot       0.67      0.71      0.69        14
            Peach___healthy       0.90      1.00      0.95        18
           Potato___healthy       0.95      0.90      0.92        20
   Strawberry___Leaf_scorch       1.00      0.86      0.92        21
    Tomato___Bacterial_spot       0.92      0.88      0.90        25
           Tomato___healthy       0.96      1.00      0.98        22

                   accuracy                           0.91       140
                  macro avg       0.91      0.91      0.90       140
               weighted avg       0.92      0.91      0.91     